<img src=".\images\Logo.png" width=150 align="left" /> <img src=".\images\Logo2.jpg" width=450 align="right" />


# <center><font color= #1e8449 > <b>CU25_Modelo de gestión de Lista de Espera Quirúrgica</font></center>

<font color='steelblue'><b>Citizenlab Data Science Methodology > III - Feature Engineering Domain </font>
***
> # <font color='steelblue'> <b>14.- Feature Data Transform</font>

<font color='steelblue'>Feature Data Transform is the process that allows change (if is required) the type and/or distribution of data features (e.g. scaling, normalizing o standardizing data features).</font>


## <font color='steelblue'>Tasks</font>

<font color='steelblue'><b>Perform Basic Data Transforms						
    
<font color='steelblue'><b>Perform Categorical Variable Transformation						
- Encode Transformation					
    - One-hot encoding				
    - Ordinal encoding				
    - Dummy encoding				
- Evaluate a Logistic Regression model					
- Consider Embedding if text mining context					
    
<font color='steelblue'><b>Perform Numeric Variable Transformation						
- Scale Transformation					
    - Normalization				
    - Standardization				
    - IQR Robust Scaler Transform				
    - Evaluate a KNN model				
- Distribution Transformation					
    - Discretization				
        - Uniform			
        - Clustered(k-Means)			
        - Quantile			
            - Normal Quantile		
            - Uniform Quantile		
            - Evaluate a KNN model		
        - Evaluate a KNN model			
    - Power transforms (Make Distributions More Gaussian)				
        - Box-Cox Transform			
        - Yeo-Johnson Transform			
        - Evaluate a KNN model			


## Consideraciones casos CitizenLab programados en R

* Algunas de las tareas de este proceso se han realizado en los notebooks del proceso 05 Data Collection porque eran necesarias para las tareas ETL. En esos casos, en este notebook se referencia al notebook del proceso 05 correspondiente
* Otras tareas típicas de este proceso se realizan en los notebooks del dominio IV al ser más eficiente realizarlas en el propio pipeline de modelización.
* Por tanto en los notebooks de este proceso de manera general se incluyen las comprobaciones necesarias, y comentarios si procede
* Las tareas del proceso se van a aplicar solo a los archivos que forman parte del despliegue, ya que hay muchos archivos intermedios que no procede pasar por este proceso
* El nombre de archivo del notebook hace referencia al nombre de archivo del proceso 05 al que se aplica este proceso, por eso pueden no ser correlativa la numeración
* Las comprobaciones se van a realizar teniendo en cuenta que el lenguaje utilizado en el despliegue de este caso es R

## <font color='green'>File</font>

- <font color='steelblue'> <b>Input File: CU_25_09.2_01_lista_espera_completo_clean_v_01.csv</font>
- <font color='steelblue'> <b>Output File: No aplica</font>

### Encoding

Con la siguiente expresión se evitan problemas con el encoding al ejecutar el notebook. Es posible que deba ser eliminada o adaptada a la máquina en la que se ejecute el código.

In [51]:
Sys.setlocale(category = "LC_ALL", locale = "es_ES.UTF-8")

[1] "LC_COLLATE=es_ES.UTF-8;LC_CTYPE=es_ES.UTF-8;LC_MONETARY=es_ES.UTF-8;LC_NUMERIC=C;LC_TIME=es_ES.UTF-8"

## <font color='green'>Settings</font>

### Libraries to use

In [52]:
library(readr)
library(dplyr)
library(tidyr)
library(forcats)
library(lubridate)


### Paths

In [53]:
iPath <- "Data/Input/"
oPath <- "Data/Output/"

## <font color='green'>Data Load</font>

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if using this option

In [54]:
# file_data <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [55]:
iFile <- "CU_25_09.2_01_lista_espera_completo_clean_v_01.csv"
file_data <- paste0(iPath, iFile)

if(file.exists(file_data)){
    cat("Se leerán datos del archivo: ", file_data)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leerán datos del archivo:  Data/Input/CU_25_09.2_01_lista_espera_completo_clean_v_01.csv

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [56]:
data <- read.csv(file_data)


Estructura de  los datos:

In [57]:
data |> glimpse()

Rows: 55,216
Columns: 46
$ Hospital          <chr> "HOSPITAL REY JUAN CARLOS", "HOSPITAL CENTRAL DE LA …
$ Especialidad      <chr> "UROLOGÍA", "ODONTOESTOMATOLOGÍA", "GINECOLOGÍA", "D…
$ total_pacientes   <int> 344, 0, 52, 37, 0, 4, 0, 718, 0, 271, 108, 0, 34, 86…
$ ano               <int> 2021, 2020, 2021, 2021, 2021, 2020, 2021, 2020, 2021…
$ semana            <int> 30, 36, 49, 23, 3, 5, 50, 7, 35, 1, 42, 10, 21, 33, …
$ CODCNH            <int> 281348, 280724, 281292, 281292, 281236, 280724, 2807…
$ id_area           <int> 8, 7, 11, 11, 11, 7, 3, 6, 1, 2, 2, 8, 11, 11, 1, 3,…
$ nombre_area       <chr> "SUR-OESTE I", "CENTRO-OESTE", "SUR II", "SUR II", "…
$ cmunicipio        <int> 280920, 280796, 280133, 280133, 281610, 280796, 2800…
$ Municipio         <chr> "MÓSTOLES", "MADRID", "ARANJUEZ", "ARANJUEZ", "VALDE…
$ CAMAS             <int> 382, 475, 98, 98, 182, 475, 507, 613, 269, 1143, 156…
$ Clase             <chr> "HOSPITALES GENERALES", "HOSPITALES GENERALES", "HOS…
$ Dependencia  

Muestra de los primeros datos:

In [58]:
data |> slice_head(n = 5)

Hospital,Especialidad,total_pacientes,ano,semana,CODCNH,id_area,nombre_area,cmunicipio,Municipio,⋯,t9_1,t10_1,t11_1,t12_1,capacidad,pacientes,consultas,hospitalizaciones,Target,is_train
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<lgl>
HOSPITAL REY JUAN CARLOS,UROLOGÍA,344,2021,30,281348,8,SUR-OESTE I,280920,MÓSTOLES,⋯,0.3348780,0.1369254,0.5072726,0.5849309,17,1447,573,12,54.45,TRUE
HOSPITAL CENTRAL DE LA DEFENSA GOMEZ ULLA,ODONTOESTOMATOLOGÍA,0,2020,36,280724,7,CENTRO-OESTE,280796,MADRID,⋯,0.4914365,0.1217100,0.4915713,0.5597213,0,1211,45,0,0.00,TRUE
HOSPITAL UNIVERSITARIO DEL TAJO,GINECOLOGÍA,52,2021,49,281292,11,SUR II,280133,ARANJUEZ,⋯,0.3346063,0.1517321,0.5024130,0.5900028,8,1293,108,2,37.96,TRUE
HOSPITAL UNIVERSITARIO DEL TAJO,DERMATOLOGÍA,37,2021,23,281292,11,SUR II,280133,ARANJUEZ,⋯,0.3346063,0.1517321,0.5024130,0.5900028,5,1501,103,2,23.14,TRUE
HOSPITAL UNIVERSITARIO INFANTA ELENA,ODONTOESTOMATOLOGÍA,0,2021,3,281236,11,SUR II,281610,VALDEMORO,⋯,0.3346063,0.1517321,0.5024130,0.5900028,0,1240,44,0,0.00,TRUE


## Basic Data Transforms

### Data Selecting

In [59]:
data |> select(1)

Hospital
<chr>
HOSPITAL REY JUAN CARLOS
HOSPITAL CENTRAL DE LA DEFENSA GOMEZ ULLA
HOSPITAL UNIVERSITARIO DEL TAJO
HOSPITAL UNIVERSITARIO DEL TAJO
HOSPITAL UNIVERSITARIO INFANTA ELENA
HOSPITAL CENTRAL DE LA DEFENSA GOMEZ ULLA
HOSPITAL UNIVERSITARIO PRINCIPE DE ASTURIAS
HOSPITAL UNIVERSITARIO PUERTA DE HIERRO MAJADAHONDA
HOSPITAL UNIVERSITARIO INFANTA LEONOR


### Data Filtering

In [60]:
data |> filter(ano == 2021) |> select(1)

Hospital
<chr>
HOSPITAL REY JUAN CARLOS
HOSPITAL UNIVERSITARIO DEL TAJO
HOSPITAL UNIVERSITARIO DEL TAJO
HOSPITAL UNIVERSITARIO INFANTA ELENA
HOSPITAL UNIVERSITARIO PRINCIPE DE ASTURIAS
HOSPITAL UNIVERSITARIO INFANTA LEONOR
HOSPITAL UNIVERSITARIO SANTA CRISTINA
HOSPITAL UNIVERSITARIO 12 DE OCTUBRE
HOSPITAL UNIVERSITARIO DEL SURESTE


### Insert New Column

In [61]:
data |>
    mutate(x = TRUE) |> head()

,Hospital,Especialidad,total_pacientes,ano,semana,CODCNH,id_area,nombre_area,cmunicipio,Municipio,⋯,t10_1,t11_1,t12_1,capacidad,pacientes,consultas,hospitalizaciones,Target,is_train,x
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<lgl>,<lgl>
1,HOSPITAL REY JUAN CARLOS,UROLOGÍA,344,2021,30,281348,8,SUR-OESTE I,280920,MÓSTOLES,⋯,0.1369254,0.5072726,0.5849309,17,1447,573,12,54.45,TRUE,TRUE
2,HOSPITAL CENTRAL DE LA DEFENSA GOMEZ ULLA,ODONTOESTOMATOLOGÍA,0,2020,36,280724,7,CENTRO-OESTE,280796,MADRID,⋯,0.1217100,0.4915713,0.5597213,0,1211,45,0,0.00,TRUE,TRUE
3,HOSPITAL UNIVERSITARIO DEL TAJO,GINECOLOGÍA,52,2021,49,281292,11,SUR II,280133,ARANJUEZ,⋯,0.1517321,0.5024130,0.5900028,8,1293,108,2,37.96,TRUE,TRUE
4,HOSPITAL UNIVERSITARIO DEL TAJO,DERMATOLOGÍA,37,2021,23,281292,11,SUR II,280133,ARANJUEZ,⋯,0.1517321,0.5024130,0.5900028,5,1501,103,2,23.14,TRUE,TRUE
5,HOSPITAL UNIVERSITARIO INFANTA ELENA,ODONTOESTOMATOLOGÍA,0,2021,3,281236,11,SUR II,281610,VALDEMORO,⋯,0.1517321,0.5024130,0.5900028,0,1240,44,0,0.00,TRUE,TRUE
6,HOSPITAL CENTRAL DE LA DEFENSA GOMEZ ULLA,CIRUGÍA TORÁCICA,4,2020,5,280724,7,CENTRO-OESTE,280796,MADRID,⋯,0.1217100,0.4915713,0.5597213,5,1504,42,1,6.25,TRUE,TRUE


### Delete Column

In [62]:
data |> select(-t4_2) |> head()

,Hospital,Especialidad,total_pacientes,ano,semana,CODCNH,id_area,nombre_area,cmunicipio,Municipio,⋯,t9_1,t10_1,t11_1,t12_1,capacidad,pacientes,consultas,hospitalizaciones,Target,is_train
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<lgl>
1,HOSPITAL REY JUAN CARLOS,UROLOGÍA,344,2021,30,281348,8,SUR-OESTE I,280920,MÓSTOLES,⋯,0.3348780,0.1369254,0.5072726,0.5849309,17,1447,573,12,54.45,TRUE
2,HOSPITAL CENTRAL DE LA DEFENSA GOMEZ ULLA,ODONTOESTOMATOLOGÍA,0,2020,36,280724,7,CENTRO-OESTE,280796,MADRID,⋯,0.4914365,0.1217100,0.4915713,0.5597213,0,1211,45,0,0.00,TRUE
3,HOSPITAL UNIVERSITARIO DEL TAJO,GINECOLOGÍA,52,2021,49,281292,11,SUR II,280133,ARANJUEZ,⋯,0.3346063,0.1517321,0.5024130,0.5900028,8,1293,108,2,37.96,TRUE
4,HOSPITAL UNIVERSITARIO DEL TAJO,DERMATOLOGÍA,37,2021,23,281292,11,SUR II,280133,ARANJUEZ,⋯,0.3346063,0.1517321,0.5024130,0.5900028,5,1501,103,2,23.14,TRUE
5,HOSPITAL UNIVERSITARIO INFANTA ELENA,ODONTOESTOMATOLOGÍA,0,2021,3,281236,11,SUR II,281610,VALDEMORO,⋯,0.3346063,0.1517321,0.5024130,0.5900028,0,1240,44,0,0.00,TRUE
6,HOSPITAL CENTRAL DE LA DEFENSA GOMEZ ULLA,CIRUGÍA TORÁCICA,4,2020,5,280724,7,CENTRO-OESTE,280796,MADRID,⋯,0.4914365,0.1217100,0.4915713,0.5597213,5,1504,42,1,6.25,TRUE


### Rank Data

<font color='darkgrey'><b>Operation</font>

In [63]:
data |> mutate(rank = order(capacidad)) |> head()

,Hospital,Especialidad,total_pacientes,ano,semana,CODCNH,id_area,nombre_area,cmunicipio,Municipio,⋯,t10_1,t11_1,t12_1,capacidad,pacientes,consultas,hospitalizaciones,Target,is_train,rank
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<lgl>,<int>
1,HOSPITAL REY JUAN CARLOS,UROLOGÍA,344,2021,30,281348,8,SUR-OESTE I,280920,MÓSTOLES,⋯,0.1369254,0.5072726,0.5849309,17,1447,573,12,54.45,TRUE,2
2,HOSPITAL CENTRAL DE LA DEFENSA GOMEZ ULLA,ODONTOESTOMATOLOGÍA,0,2020,36,280724,7,CENTRO-OESTE,280796,MADRID,⋯,0.1217100,0.4915713,0.5597213,0,1211,45,0,0.00,TRUE,5
3,HOSPITAL UNIVERSITARIO DEL TAJO,GINECOLOGÍA,52,2021,49,281292,11,SUR II,280133,ARANJUEZ,⋯,0.1517321,0.5024130,0.5900028,8,1293,108,2,37.96,TRUE,31
4,HOSPITAL UNIVERSITARIO DEL TAJO,DERMATOLOGÍA,37,2021,23,281292,11,SUR II,280133,ARANJUEZ,⋯,0.1517321,0.5024130,0.5900028,5,1501,103,2,23.14,TRUE,40
5,HOSPITAL UNIVERSITARIO INFANTA ELENA,ODONTOESTOMATOLOGÍA,0,2021,3,281236,11,SUR II,281610,VALDEMORO,⋯,0.1517321,0.5024130,0.5900028,0,1240,44,0,0.00,TRUE,50
6,HOSPITAL CENTRAL DE LA DEFENSA GOMEZ ULLA,CIRUGÍA TORÁCICA,4,2020,5,280724,7,CENTRO-OESTE,280796,MADRID,⋯,0.1217100,0.4915713,0.5597213,5,1504,42,1,6.25,TRUE,52


## Categorical Variable Transformation

### Encode Transformation

No aplica

#### Ordinal Encoding Transform

No aplica

#### One Hot Encoding Transform

#### Dummy Variable Encoding Transform

### Embedding Transformation

Specific encode for text mining context. No code here.

## Numeric Variable Transformation: Scale

### Data to Transform

#### Evaluating Normalization Tranform

#### Evaluating Standarization Transform

### Normalization Transform

No aplica

### Standarization Transform

No aplica

## Numeric Variable Transformation: Distribution

No aplica

###	Discretization Transform

##### Evaluating Discretization Transformations

#### Uniform Discretization Transform

No aplica

### Power Transform

No aplica

#### Data to Transform

No aplica

##### Evaluating Box-Cox tranform

##### Evaluating Yeo-Johnson tranform

#### Box-Cox Transform

No aplica

#### Yeo-Johnson Transform

No aplica

## <font color='green'>Data Save</font>

* Solo si se han hecho cambios

* No aplica

<font color='tomato'> Identificamos los datos a guardar</font>

In [64]:
data_to_save <- data


<font color='tomato'>Estructura de nombre de archivos:</font>

* Código del caso de uso, por ejemplo "CU_04"
* Número del proceso que lo genera, por ejemplo "_06".
* Resto del nombre del archivo de entrada
* Extensión del archivo

Ejemplo: "CU_04_06_01_01_zonasgeo.json, primer fichero que se genera en la tarea 01 del proceso 05 (Data Collection) para el caso de uso 04 (vacunas) y que se ha transformado en el proceso 06

Importante mantener los guiones bajos antes de proceso, tarea, archivo y nombre

### Proceso 14

In [65]:
# caso <- "CU_XX"
# proceso <- '_09.2'
# tarea <- "_XX"
# archivo <- ""
# proper <- "_xxxxx"
# extension <- ".csv"

<font color='tomato'><b> OPCION A:</b> Uso del paquete "tcltk" para mayor comodidad</font>

* Buscar carpeta, escribir nombre de archivo SIN extensión (se especifica en el código)
* Especificar sufijo2 si es necesario
* Cambiar datos por datos_xx si es necesario

In [66]:
# file_save <- paste0(caso, proceso, tarea, tcltk::tkgetSaveFile(), proper, extension) 
# path_out <- paste0(oPath, file_save)
# write_csv(data_to_save_xxxxx, path_out)

# cat('File saved as: ')
# path_out

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>

- Los ficheros de salida del proceso van siempre a Data/Output/.  

In [ ]:
# file_save <- paste0(caso, proceso, tarea, archivo, proper, extension) 
# path_out <- paste0(oPath, file_save)
# write_csv(data_to_save_xxxxx, path_out)

# cat('File saved as: ')
# path_out

#### Copia del fichero a Input

Si el archivo se va a usar en otros notebooks, copiar a la carpeta Input

In [ ]:
# path_in <- paste0(iPath, file_save)
# file.copy(path_out, path_in, overwrite = TRUE)

## <font color='#2874a6'>REPORT</font>

A continuación se realizará un informe de las acciones realizadas

## <font color=' #2874a6 '>Main Actions Carried Out</font>

- Si eran necesarias se han realizado en el proceso 05 por cuestiones de eficiencia
- O bien se hacen en el dominio IV o V para integrar en el pipeline de modelización

## <font color=' #2874a6 '>Main Conclusions</font>

- Los datos están listos para la modelización y despliegue


## <font color='RED'>CODE TO DEPLOY (PILOT)</font>

A continuación se incluirá el código que deba ser llevado a despliegue para producción, dado que se entiende efectúa operaciones necesarias sobre los datos en la ejecución del prototipo

<font color=' red '><b>Description</b></font>

- No hay nada que desplegar en el piloto, ya que estos datos son estáticos o en todo caso cambian con muy poca frecuencia, altamente improbable durante el proyecto.


<font color=' red '><b>CODE</b></font>